In [6]:
# install requirements for colab
import sys
if 'google.colab' in sys.modules:
    !pip install -r requirements.txt

In [7]:
from models.tinyyolov2 import TinyYoloV2
print(TinyYoloV2)

<class 'models.tinyyolov2.TinyYoloV2'>
